In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv('C:/Users/user/Desktop/Data Science or Data Analytics/Github/Data-Analytics-on-Chips-Category/cleaned_dataset.csv')

In [3]:
df.head()

,DATE,STORE_NBR,LYLTY_CARD_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE(g),BRAND_NAME,LIFESTAGE,PREMIUM_CUSTOMER,Month
0,2018-07-03,19,19205,16466,26,Pringles Sweet&Spcy BBQ 134g,1,3.7,134,Pringles,OLDER SINGLES/COUPLES,Mainstream,2018-07
1,2018-07-03,189,189381,190189,84,GrnWves Plus Btroot & Chilli Jam 180g,1,3.1,180,Grain,OLDER FAMILIES,Mainstream,2018-07
2,2018-07-03,124,124236,127984,104,Infuzions Thai SweetChili PotatoMix 110g,1,3.8,110,Infuzions,OLDER FAMILIES,Budget,2018-07
3,2018-07-03,70,70131,68241,60,Kettle Tortilla ChpsFeta&Garlic 150g,2,9.2,150,Kettle,RETIREES,Premium,2018-07
4,2018-07-03,33,33140,30342,10,RRD SR Slow Rst Pork Belly 150g,2,5.4,150,RRD,YOUNG FAMILIES,Mainstream,2018-07


In [4]:
# Change the dtype for date to datetime format
df['DATE'] = pd.to_datetime(df['DATE'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264836 entries, 0 to 264835
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   DATE              264836 non-null  datetime64[ns]
 1   STORE_NBR         264836 non-null  int64         
 2   LYLTY_CARD_NBR    264836 non-null  int64         
 3   TXN_ID            264836 non-null  int64         
 4   PROD_NBR          264836 non-null  int64         
 5   PROD_NAME         264836 non-null  object        
 6   PROD_QTY          264836 non-null  int64         
 7   TOT_SALES         264836 non-null  float64       
 8   PACK_SIZE(g)      264836 non-null  int64         
 9   BRAND_NAME        264836 non-null  object        
 10  LIFESTAGE         264836 non-null  object        
 11  PREMIUM_CUSTOMER  264836 non-null  object        
 12  Month             264836 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(6), object(5)
memor

# GETTING THE METRICS THAT'LL BE USED TO FIND THE CONTROL STORES

METRICS TO FIND:
* total sales revenue* total number of customers
* Average number of transactions per customer

Consider the monthly sales experience of eachcustomer


In [7]:
def create_metrics(df):

    # Grouping by STORE_NBR and month (M) from the DATE column
    df['Month'] = df['DATE'].dt.to_period('M')
    
    # Total sales revenue
    T_sales = df.groupby([ 'Month','STORE_NBR'])['TOT_SALES'].sum().reset_index()

    # Total number of customers
    T_Cust = df.groupby(['Month', 'STORE_NBR'])['LYLTY_CARD_NBR'].nunique().reset_index()

    # Average number of transactions per customer
    Avg_Tcust =  df.groupby(['Month', 'STORE_NBR'])['LYLTY_CARD_NBR'].count() / \
                df.groupby(['Month', 'STORE_NBR'])['LYLTY_CARD_NBR'].nunique()
    #print(Avg_Tcust)
    
   # Merge the metrics while considering the months
    result1 = pd.merge(T_sales, T_Cust, how='outer', on=[ 'Month', 'STORE_NBR'])
    #print(result1.columns)
    result2 =  pd.merge(result1, Avg_Tcust, how='outer', on=['Month','STORE_NBR'])
    #print(result2.columns)  # Check the actual number of columns

   # Clarifying the name of the columns;
    result2.columns = [ 'Month', 'STORE_NBR', 'M_SalesRevenue', 'M_NumberOfCustomers', 'M_AvgTranscPerCustomers']
    
    return result2

Metrics = create_metrics(df)
metrics = pd.DataFrame(Metrics)
metrics


,Month,STORE_NBR,M_SalesRevenue,M_NumberOfCustomers,M_AvgTranscPerCustomers
0,2018-07,1,183.4,46,1.043478
1,2018-07,2,141.2,36,1.055556
2,2018-07,3,1140.1,106,1.235849
3,2018-07,4,1326.5,122,1.245902
4,2018-07,5,776.2,90,1.277778
...,...,...,...,...,...
3411,2019-07,268,10.4,2,1.000000
3412,2019-07,269,65.2,8,1.000000
3413,2019-07,270,32.2,4,1.000000
3414,2019-07,271,76.4,11,1.000000


# SELECT CONTROL STORES
#### USING THE METRICS TO FIND THE CONTROL STORES FOR THE GIVEN TRIAL STORES

Trial Stores: 77, 86 and 88
* Write a function* Consider using Pearson correlatio
*  Or metric such as a magnitude distance e.g Observed distance - minimum distance)/(Maximum distance - minimum distance) as a measre.

My ideas:

* Speicify the trial stores in the function.
* Create another dataframe with just the trial stores and their respective metrics present
* Create another dataframe that excludes the trial stores within the function.
* Use Pearson correlation to measure the relationship between the trial stores metrics and the other stores metrics
* * Aggregate metrics by Month i.e calculate the monthly average of your three metrics for both groups
  * Merge the dataframes for comparison
  * Compute Pearson correlation between the trial stores and other stores for each metrics.

In [80]:
# Defining the trial stores


def find_controlstores(metrics, corr_min= 0.85, corr_max=0.95):
    
    trial_stores = [77, 86, 88]   # Defined trial stores
    control_stores = {} # Dictionary to store selected control stores
    
    # Creating separate dataframes
    trial_df = metrics[metrics['STORE_NBR'].isin(trial_stores)]
    
    other_stores = metrics[~metrics['STORE_NBR'].isin(trial_stores)]
    
    
    # Aggregate metrics by Month
    trial_metrics = trial_df.groupby(['Month', 'STORE_NBR'])[['M_SalesRevenue', 'M_NumberOfCustomers', 'M_AvgTranscPerCustomers']].mean()
    #print(trial_metrics)
    
    # Aggregate metrics for other stores
    other_metrics = other_stores.groupby(['Month', 'STORE_NBR'])[['M_SalesRevenue', 'M_NumberOfCustomers', 'M_AvgTranscPerCustomers']].mean()
    #print(other_metrics)

    # Pivot the data to have stores as rows and metrics over time as columns
    trial_pivot = trial_metrics.unstack(level=1)
    other_pivot = other_metrics.unstack(level=1)

    for trial_store in trial_stores:
        print(f"\n### Correlation for Trial Store {trial_store} ###")

        # Extract data for the current trial store
        trial_store_metrics = trial_pivot.xs(trial_store, axis=1, level=1)
        
        # Compute correlation with all other stores
        correlations = other_pivot.corrwith(trial_store_metrics, axis=1)
        
        # Convert to a dataframe for better readavitlity
        corr_df = correlations.reset_index()
        corr_df.columns = ['STORE_NBR', 'Correlation']

        # Filter stores within the giben correlation range
        suitable_stores = corr_df[(corr_df['Correlation'] >= corr_min) & (corr_df['Correlation'] <= corr_max)]

        # Store selected control stores for this trial store
        control_stores[trial_store] = suitable_stores['STORE_NBR'].tolist()

        # Display correlations
        print(suitable_stores)

    return control_stores  # Returns a dictionary of selected control stores
                                   

    '''
    # Merge trial and other stores' metrics on Month
    merged_metrics = trial_metrics.merge(other_metrics, on="Month", suffixes=("_trial", "_other"))

    # Compute Pearson correlation for each metric
    correlations = merged_metrics.corr(method='pearson')

    # Display correlations between trial and other stores
    print(correlations.loc[['M_SalesRevenue_trial', 'M_NumberOfCustomers_trial', 'M_AvgTranscPerCustomers_trial'],
          ['M_SalesRevenue_other', 'M_NumberOfCustomers_other', 'M_AvgTranscPerCustomers_other']])

    #return correlations
    '''
print('\n### Final Selected Control Stores ###')
print(find_controlstores(metrics))    



### Final Selected Control Stores ###

### Correlation for Trial Store 77 ###


ValueError: cannot join with no overlapping index names

In [ ]:
sns.regplot ( x = 'STORE_NBR', y = 'M_SalesRevenue', data = metrics)
plt.ylim(0,)